# Module 2: Building DevAssist - Feature Implementer

## From Analysis to Action

In Module 1, we built the **Project Analyzer** that can understand any codebase. Now we'll extend DevAssist to actually **implement features** - making real changes to code with tracking and validation.

### What We're Building

| Module | Component | Capability |
|--------|-----------|------------|
| Module 1 | Project Analyzer | Understand any codebase |
| **Module 2** | **Feature Implementer** | **Make code changes with tracking** |
| Module 3 | Full Dev Workflow | Multi-agent research, implement, test, document |

### This Module's Goals

1. Observe the agent loop in detail
2. Plan implementations with `TodoWrite`
3. Modify code using `Edit` and `Bash`
4. Use sessions for multi-turn development
5. Create an "implement-feature" skill

### Continuing with TaskFlow

We'll continue working with the TaskFlow project from Module 1, adding a new feature: **task search functionality**.

---

## 1. Setup

In [ ]:
import os
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass("Enter your Anthropic API key: ")

In [ ]:
from claude_agent_sdk import (
    query,
    ClaudeAgentOptions,
    AssistantMessage,
    ToolUseMessage,
    ToolResultMessage,
    ResultMessage
)

PROJECT_PATH = os.path.abspath("../sample_project")
print(f"Working with: {PROJECT_PATH}")

---

## 2. Exercise 1: Observing the Agent Loop

Before we make changes, let's deeply understand how the agent works. The loop follows this pattern:

```
User Query -> Reasoning -> Tool Call -> Tool Result -> Reasoning -> ... -> Final Response
```

### Building a Detailed Observer

In [ ]:
async def observe_agent_loop(prompt: str, project_path: str):
    """Watch every step of the agent loop."""
    
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Glob", "Grep"],
        permission_mode="bypassPermissions"
    )
    
    step = 0
    
    print(f"{'='*70}")
    print(f"TASK: {prompt}")
    print(f"{'='*70}\n")
    
    async for message in query(prompt=f"{prompt}\n\nProject: {project_path}", options=options):
        step += 1
        
        if isinstance(message, AssistantMessage):
            print(f"[{step}] REASONING")
            if message.content:
                preview = message.content[:150] + "..." if len(message.content) > 150 else message.content
                print(f"    {preview}")
            print()
            
        elif isinstance(message, ToolUseMessage):
            print(f"[{step}] TOOL CALL: {message.tool_name}")
            print(f"    Input: {str(message.tool_input)[:80]}...")
            print()
            
        elif isinstance(message, ToolResultMessage):
            status = "SUCCESS" if message.success else "FAILED"
            print(f"[{step}] TOOL RESULT: {status}")
            print()
            
        elif isinstance(message, ResultMessage):
            print(f"[{step}] SESSION COMPLETE")
            print(f"    Session ID: {message.session_id[:20]}...")
    
    print(f"\n{'='*70}")
    print(f"Total steps in loop: {step}")
    print(f"{'='*70}")

In [ ]:
# Watch the agent analyze TaskFlow
await observe_agent_loop(
    "Find the main entry point and list all available CLI commands",
    PROJECT_PATH
)

### Understanding the Loop

| Step Type | What Happens | Your Code Can... |
|-----------|--------------|------------------|
| `AssistantMessage` | Claude thinks/responds | Show reasoning to user |
| `ToolUseMessage` | About to use a tool | Log, validate, or block |
| `ToolResultMessage` | Tool finished | Check success, log output |
| `ResultMessage` | Session complete | Save session ID |

---

## 3. Exercise 2: Implementation Planning with TodoWrite

Before making changes, a good practice is to plan. The `TodoWrite` tool helps agents track multi-step tasks.

### The Feature We're Adding

Let's add a **search command** to TaskFlow that lets users search tasks by keyword:

```bash
python task_manager.py search "groceries"
```

In [ ]:
async def plan_feature(feature_description: str, project_path: str):
    """Plan a feature implementation using TodoWrite."""
    
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Glob", "Grep", "TodoWrite"],
        permission_mode="bypassPermissions"
    )
    
    prompt = f"""You are DevAssist. Plan the implementation of this feature:
    
    FEATURE: {feature_description}
    PROJECT: {project_path}
    
    Steps:
    1. First, analyze the existing codebase to understand the patterns
    2. Use TodoWrite to create a detailed implementation plan
    3. List each file that needs to be modified and what changes are needed
    
    Be specific about what code needs to be added/modified."""
    
    plan_result = {"todos": [], "analysis": ""}
    
    print("DevAssist is planning the implementation...\n")
    
    async for message in query(prompt=prompt, options=options):
        if isinstance(message, ToolUseMessage):
            if message.tool_name == "TodoWrite":
                print(f"Creating implementation plan...")
                plan_result["todos"] = message.tool_input
            else:
                print(f"  [{message.tool_name}] Analyzing...")
                
        elif isinstance(message, AssistantMessage) and message.content:
            plan_result["analysis"] = message.content
            
        elif isinstance(message, ResultMessage):
            plan_result["session_id"] = message.session_id
    
    return plan_result

In [ ]:
# Plan our search feature
plan = await plan_feature(
    "Add a 'search' command that searches tasks by keyword in title",
    PROJECT_PATH
)

print("\n" + "="*60)
print("IMPLEMENTATION PLAN")
print("="*60 + "\n")
print(plan["analysis"])

---

## 4. Exercise 3: Making Code Changes

Now let's actually implement the feature. We'll use `Edit` to modify files.

### Permission Modes for Editing

| Mode | Behavior | Use Case |
|------|----------|----------|
| `default` | Ask for each edit | Development, learning |
| `acceptEdits` | Auto-approve edits | Trusted code tasks |
| `bypassPermissions` | No prompts | CI/CD, automation |

In [ ]:
async def implement_feature(feature_description: str, project_path: str):
    """Implement a feature with code changes."""
    
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Glob", "Grep", "Edit", "TodoWrite"],
        permission_mode="acceptEdits"  # Auto-approve edits
    )
    
    prompt = f"""You are DevAssist. Implement this feature:
    
    FEATURE: {feature_description}
    PROJECT: {project_path}
    
    Requirements:
    1. First understand the existing code patterns
    2. Create a todo list to track your progress
    3. Make the necessary code changes
    4. Follow the existing code style
    5. Mark todos as complete as you finish each step
    
    Implement the feature now."""
    
    changes = []
    
    print("DevAssist is implementing the feature...\n")
    
    async for message in query(prompt=prompt, options=options):
        if isinstance(message, ToolUseMessage):
            if message.tool_name == "Edit":
                file_path = message.tool_input.get("file_path", "unknown")
                print(f"  [EDIT] Modifying: {file_path}")
                changes.append(file_path)
            elif message.tool_name == "TodoWrite":
                print(f"  [TODO] Updating progress...")
            else:
                print(f"  [{message.tool_name}] Working...")
                
        elif isinstance(message, ToolResultMessage):
            if not message.success:
                print(f"  [WARNING] Tool failed!")
                
        elif isinstance(message, AssistantMessage) and message.content:
            # Final summary
            pass
            
        elif isinstance(message, ResultMessage):
            session_id = message.session_id
    
    print(f"\nImplementation complete!")
    print(f"Files modified: {len(changes)}")
    for f in changes:
        print(f"  - {f}")
    
    return {"changes": changes, "session_id": session_id}

In [ ]:
# Implement the search feature
result = await implement_feature(
    """Add a 'search' command to task_manager.py that:
    1. Takes a keyword argument
    2. Searches task titles for the keyword (case-insensitive)
    3. Returns matching tasks
    Also add a search_tasks method to the TaskStorage class in storage.py""",
    PROJECT_PATH
)

### Verify the Changes

In [ ]:
# Check what was added
async def verify_changes(project_path: str):
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Grep"],
        permission_mode="bypassPermissions"
    )
    
    async for msg in query(
        prompt=f"""Look at the project at {project_path} and:
        1. Find the new 'search' command in task_manager.py
        2. Find the search_tasks method in storage.py
        3. Show me the new code that was added""",
        options=options
    ):
        if isinstance(msg, AssistantMessage) and msg.content:
            print(msg.content)

await verify_changes(PROJECT_PATH)

---

## 5. Exercise 4: Sessions for Multi-Turn Development

Real development is iterative. Sessions let you continue a conversation with context preserved.

### Session Workflow

```
Query 1: "Add search feature"  ->  Session ID: abc123
                                          |
Query 2: "Add tests for it"    <-  Resume: abc123
                                          |
Query 3: "Fix the bug"         <-  Resume: abc123
```

In [ ]:
class DevAssistSession:
    """Manage a multi-turn development session."""
    
    def __init__(self, project_path: str):
        self.project_path = project_path
        self.session_id = None
        self.history = []
    
    async def chat(self, message: str, allow_edits: bool = False):
        """Send a message and get a response."""
        
        tools = ["Read", "Glob", "Grep"]
        if allow_edits:
            tools.extend(["Edit", "TodoWrite"])
        
        options = ClaudeAgentOptions(
            allowed_tools=tools,
            permission_mode="acceptEdits" if allow_edits else "bypassPermissions",
            resume=self.session_id  # Resume if we have a session
        )
        
        prompt = f"{message}\n\nProject: {self.project_path}"
        
        response = ""
        
        async for msg in query(prompt=prompt, options=options):
            if isinstance(msg, AssistantMessage) and msg.content:
                response = msg.content
            elif isinstance(msg, ToolUseMessage):
                print(f"  [{msg.tool_name}]...")
            elif isinstance(msg, ResultMessage):
                self.session_id = msg.session_id
        
        self.history.append({"user": message, "assistant": response[:200]})
        return response

In [ ]:
# Start a development session
session = DevAssistSession(PROJECT_PATH)

# First query - analyze
print("=== Query 1: Analyze ===")
response = await session.chat("What commands does TaskFlow currently support?")
print(response[:500])
print(f"\nSession ID: {session.session_id[:20]}...")

In [ ]:
# Second query - follow up (session resumed automatically)
print("\n=== Query 2: Follow-up ===")
response = await session.chat("Which of those commands uses the storage module?")
print(response[:500])

In [ ]:
# Third query - suggest improvement
print("\n=== Query 3: Suggest ===")
response = await session.chat("What's one feature that would make TaskFlow better?")
print(response[:500])

---

## 6. Exercise 5: Creating the "implement-feature" Skill

Let's create a reusable skill that automatically activates when users want to implement features.

### Skill Structure

In [ ]:
import os

# Create skill directory
skill_dir = os.path.join(PROJECT_PATH, ".claude", "skills", "implement-feature")
os.makedirs(skill_dir, exist_ok=True)
print(f"Created: {skill_dir}")

In [ ]:
skill_content = '''---
description: Implements new features following project conventions
activation:
  - "implement"
  - "add feature"
  - "create feature"
  - "build feature"
---

# Feature Implementation Skill

You are DevAssist, an AI development partner. When implementing features:

## Process

1. **Understand** - Read existing code to understand patterns
2. **Plan** - Create a todo list with specific tasks
3. **Implement** - Make changes following existing style
4. **Verify** - Check the changes work correctly

## Code Style Rules

- Follow existing naming conventions
- Match indentation and formatting
- Add docstrings to new functions
- Keep functions focused and small

## Checklist Before Completing

- [ ] All todos marked complete
- [ ] Code follows existing patterns
- [ ] No syntax errors introduced
- [ ] Feature is documented

## Output Format

After implementation, provide:

### Changes Made
- File: [path] - [description of change]

### How to Use
[Usage example]

### Testing
[How to verify the feature works]
'''

skill_path = os.path.join(skill_dir, "skill.md")
with open(skill_path, "w") as f:
    f.write(skill_content)

print(f"Created skill at: {skill_path}")
print("\nSkill content preview:")
print(skill_content[:600] + "...")

### Test the Skill

In [ ]:
async def test_skill(project_path: str):
    """Test the implement-feature skill."""
    
    # Read skill content
    skill_path = os.path.join(project_path, ".claude", "skills", "implement-feature", "skill.md")
    with open(skill_path, "r") as f:
        skill_content = f.read()
    
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Glob", "Grep", "Edit", "TodoWrite"],
        permission_mode="acceptEdits",
        system_prompt=skill_content
    )
    
    # Test with a small feature
    async for msg in query(
        prompt=f"""Add a 'count' command to TaskFlow that shows the number of 
        pending vs completed tasks.
        
        Project: {project_path}""",
        options=options
    ):
        if isinstance(msg, AssistantMessage) and msg.content:
            print(msg.content)
        elif isinstance(msg, ToolUseMessage):
            print(f"  [{msg.tool_name}]...")

await test_skill(PROJECT_PATH)

---

## 7. Running Tests with Bash

After making changes, we should run tests. The `Bash` tool lets us execute commands.

In [ ]:
async def run_tests(project_path: str):
    """Run project tests and report results."""
    
    options = ClaudeAgentOptions(
        allowed_tools=["Bash", "Read"],
        permission_mode="bypassPermissions"
    )
    
    async for msg in query(
        prompt=f"""Run the tests for the project at {project_path}.
        Use pytest if available. Report the results.""",
        options=options
    ):
        if isinstance(msg, AssistantMessage) and msg.content:
            print(msg.content)
        elif isinstance(msg, ToolUseMessage):
            if msg.tool_name == "Bash":
                print(f"  Running: {str(msg.tool_input)[:50]}...")

await run_tests(PROJECT_PATH)

---

## 8. Cleanup

In [ ]:
# The skill is kept for future use
print(f"Skill location: {PROJECT_PATH}/.claude/skills/implement-feature/")
print("\nTo use the skill with Claude Code CLI:")
print("  cd sample_project")
print("  claude 'implement a feature to export tasks to CSV'")

---

## Summary

In this module, you built the **Feature Implementer** component of DevAssist.

### What You Learned

| Concept | Implementation |
|---------|----------------|
| Agent Loop | Observe reasoning -> tool -> result cycle |
| TodoWrite | Plan and track multi-step implementations |
| Edit Tool | Make code changes programmatically |
| Permission Modes | `acceptEdits` for trusted code changes |
| Sessions | Resume conversations with `resume=session_id` |
| Skills | Create `.claude/skills/<name>/skill.md` |

### DevAssist Can Now:

1. **Analyze** projects (Module 1)
2. **Plan** implementations with todo tracking
3. **Implement** features by editing code
4. **Continue** work across multiple sessions
5. **Run** tests to verify changes

### Coming in Module 3

We'll build the complete dev workflow with **multi-agent architecture**:
- Researcher subagent for context gathering
- Implementer subagent for code changes
- Tester subagent for validation
- Doc Writer subagent for documentation

### Quick Reference

```python
# Feature implementation agent
options = ClaudeAgentOptions(
    allowed_tools=["Read", "Glob", "Grep", "Edit", "TodoWrite"],
    permission_mode="acceptEdits"
)

# Session management
session_id = None
async for msg in query(prompt="...", options=options):
    if isinstance(msg, ResultMessage):
        session_id = msg.session_id

# Resume session
options = ClaudeAgentOptions(
    resume=session_id,
    ...
)

# Skill location
.claude/skills/<skill-name>/skill.md
```